# Умная система фильтрации отзывов - Revi AI

Интеллектуальный микросервис для анализа отзывов, который  не только классифицирует обратную связь и определяет её тональность, но и автоматически выделяет ключевые проблемы и генерирует рекомендации для их решения

## Шаг 0. Загрузка данных и их обработка

In [2]:
!pip install pymorphy2 sentence_transformers -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [64]:
from transformers import pipeline
import pandas as pd
import re
import torch
from pymorphy2 import MorphAnalyzer
import plotly.express as px
from scipy.spatial.distance import cdist
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics import accuracy_score
from pymorphy2 import MorphAnalyzer
import pandas as pd

In [65]:
import numpy as np
from torch.utils.data import DataLoader, Dataset

class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        return self.texts[idx]

def get_bert_embeddings(texts, tokenizer, model, batch_size=32):
    dataset = TextDataset(texts)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    model.eval()
    
    embeddings = []
    with torch.no_grad():
        for batch in dataloader:
            inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512)
            outputs = model(**inputs)
            cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            embeddings.append(cls_embeddings)
    
    return np.vstack(embeddings)


## Шаг 1. Классификация спам-отзывов

Определяем, спам-отзыв это или нет. Также ставим порог (0.7) для сомнительных отзывов, чтобы те сохранялись в отдельный документ и переходили на дополнительную модерацию с аннотацией

In [66]:
import torch

checkpoint = torch.load("model1.pth")

from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_config(checkpoint["config"])
model.load_state_dict(checkpoint["model_state_dict"])

tokenizer = checkpoint["tokenizer"]

print("Модель и токенизатор успешно загружены")

/tmp/ipykernel_487972/1194201951.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("model1.pth")


Модель и токенизатор успешно загружены


In [67]:
# Лемматизатор
morph = MorphAnalyzer()

def preprocess_text(text):
    # Убираем спецсимволы, переводим в нижний регистр
    text = re.sub(r"[^\w\s]", "", text.lower())
    # Исправляем пробелы и сокращения
    text = re.sub(r"\s+", " ", text).strip()
    # Лемматизация
    words = text.split()
    lemmatized_text = " ".join(morph.parse(word)[0].normal_form for word in words)
    return lemmatized_text

In [68]:
class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]

def get_bert_embeddings(texts, tokenizer, model, batch_size=32):
    dataset = TextDataset(texts)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    model.eval()

    embeddings = []
    with torch.no_grad():
        for batch in dataloader:
            inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512)
            outputs = model(**inputs)
            cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # [CLS] токен
            embeddings.append(cls_embeddings)

    return np.vstack(embeddings)

In [85]:
data = pd.read_excel('test.xlsx')
data["Текст отзыва"] = data["Текст отзыва"].apply(preprocess_text)
texts = data["Текст отзыва"].tolist()
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

from torch.nn.functional import softmax

# Устанавливаем порог уверенности
confidence_threshold = 0.6

model.eval()

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = softmax(logits, dim=-1)  # Преобразуем логиты в вероятности
    max_probs, predictions = torch.max(probabilities, dim=-1)  # Максимальная вероятность и индекс класса

# Условие для выставления класса 3 при низкой уверенности
predictions[max_probs < confidence_threshold] = 2

data["Предсказание (Спам)"] = predictions.numpy()
data["Предсказание (Спам)"] = data["Предсказание (Спам)"].replace({
    0: "Написано верно",
    1: "Спам",
    2: "Нужна модерация"
})
data


,Тип отзыва,Текст отзыва,Предсказание (Спам)
0,позитивный,спасибо вкусно частотный век но списать р нспв...,Написано верно
1,позитивный,вивщть вдвжвда выоышызд вдвжыжяб влылчост а ао...,Спам
2,позитивный,ужасный шаурм кислый фуу заказывать ещё по кие...,Написано верно
3,позитивный,верона как стандарт качество вкусно быстро сра...,Написано верно
4,позитивный,плата даа а даа садат чдатпд сжатый,Спам
...,...,...,...
169,негативный,иа интврвр киеикрн п темкм еие еие еиий еие ие...,Спам
170,негативный,yrfgfcfgvfhvghv ggyyfc ggfccbb ghccb ghhvcvv,Спам
171,негативный,очень солёный пирог с рыба к сожаление,Написано верно
172,позитивный,тататчта ьылый в твовьвтч та тула лч ч,Спам


## Шаг 2. Анализ тональности

Определяем тип отзыва - негативный / позитивный. Для негативных выставляем флаг `Спам, удалить`.

In [105]:
sentiment_pipeline = pipeline(
    "sentiment-analysis", 
    model="cardiffnlp/twitter-xlm-roberta-base-sentiment"
)

texts = data['Текст отзыва'].to_list()

def check(texts):
    results = sentiment_pipeline(texts)
    binary_results = [
        "позитивный" if result['label'] == "positive" else "негативный"
        for result in results
    ]
    return binary_results

# Анализ тональности и добавление в новый столбец
data['Тональность'] = check(texts)
data

Device set to use cpu


,Тип отзыва,Текст отзыва,Предсказание (Спам),Тональность
0,позитивный,спасибо вкусно частотный век но списать р нспв...,Написано верно,негативный
1,позитивный,вивщть вдвжвда выоышызд вдвжыжяб влылчост а ао...,Спам,негативный
2,позитивный,ужасный шаурм кислый фуу заказывать ещё по кие...,Написано верно,негативный
3,позитивный,верона как стандарт качество вкусно быстро сра...,Написано верно,позитивный
4,позитивный,плата даа а даа садат чдатпд сжатый,Спам,негативный
...,...,...,...,...
169,негативный,иа интврвр киеикрн п темкм еие еие еиий еие ие...,Спам,негативный
170,негативный,yrfgfcfgvfhvghv ggyyfc ggfccbb ghccb ghhvcvv,Спам,негативный
171,негативный,очень солёный пирог с рыба к сожаление,Написано верно,негативный
172,позитивный,тататчта ьылый в твовьвтч та тула лч ч,Спам,негативный


## Шаг 3. Анализ контекста

Если отзыв имеет тип "Негативный", то выявляем проблемы. А если отзыв "Позитивный" - причины хорошего отзыва


### zero-shot подход

In [104]:
model = SentenceTransformer('all-mpnet-base-v2')

problem_list = [
    "Доставка очень долгая",
    "Еда была холодной",
    "Курьер вел себя грубо",
    "Привезли не тот заказ",
    "Цена слишком высокая",
    "Невкусно",
    "Задержка",
    "Холодные блюда",
    "Неполная комплектация",
    "Несвежие блюда",
    "Мало начинки",
    "Долгая доставка",
    "Разваливается",
    "Не довезли",
    "Низкое качество обслуживания",
    "Сухо",
    "Не доставлен",
    "Несоответствие состава",
    "Маленькие порции",
    "Не то привезли",
    "Низкое качество продукции",
    "Инородный предмет в продукции",
    "Горелое",
    "Недоготовленное",
    "Непрезентабельный внешний вид",
    "Пересолено",
    "Проблемы при доставке",
    "Сырое",
    "Плохая транспортировка",
    "Безвкусно",
    "Переварено",
    "Жирно",
    "Плохая упаковка",
    "Кисло",
    "Несоответствие фото",
    "Пресно",
    "Недосолено",
    "Долгая обработка заказа",
    "Дорого",
    "Не учли пожелание",
    "Нет обратной связи",
    "Плохой запах",
    "Отмена заказа",
    "Несоответствие граммовки",
    "Слипшееся",
    "Другое",
    "Горько",
    "Жесткое",
    "Много риса",
    "Мятое",
    "Пережарено",
    "Грубость",
    "Проблема при оплате",
    "Взяли больше д/с",
    "Доставка не до квартиры",
    "Мало соуса",
    "Мало специй",
    "Не остро",
    "Остро",
    "Отравление",
    "Твердое",
    "Липкое",
    "Не сдали сдачу",
    "Переморожено",
    "резиновое",
    "Тонко",
    "Взяли д/с за доставку",
    "Волос",
    "Густое",
    "Курьер без термосумки",
    "Много соуса",
    "Не вернули д/c",
    "Несоответсвие состава",
    "Размякло",
    "размякло",
    "Разный размер",
    "Взяли больше д/c",
    "Вязкое",
    "Вялое",
    "Кости",
    "Курьер не ознакомлен с оплатой",
    "Курьер не ориентируется в городе",
    "Много блюд в стоп-листе",
    "Много специй",
    "мягкое",
    "Наценки",
    "Не выходят на связь",
    "Не нарезанная пицца",
    "Не обратили внимания на оплату заказа онлайн",
    "Не предоставили обещанную скидку",
    "Не предоставили чек",
    "Не разрезано",
    "Не решили проблему при доставке",
    "Не спело",
    "Не туда привезли",
    "Неаккуратно",
    "Неверная сборка заказа",
    "Неверный статус заказа",
    "Неграмотный курьер",
    "Недостоверные данные об ожидании",
    "Недостоверные данные по заказу",
    "некачественная сборка блюда",
    "неправильная подача",
    "неправильно собран заказ",
    "неравномерная начинка",
    "Неровно",
    "Нет чека",
    "неудачный выбор заправки",
    "но большие куски",
    "одинаковые",
    "отсутствие акции блюда за баллы",
    "отсутствие граммовок",
    "Перегретое",
    "плохая нарезка",
    "плохие креветки",
    "Плохой рис",
    "При звонке цены меняются",
    "привезли раньше назначенного срока",
    "разбавленный соус",
    "разбавлено",
    "Разное количество ингредиентов",
    "резиное",
    "Роллы залиты соевым соусом",
    "роллы хрустят",
    "рыхлое",
    "Слишком крупные ингедиенты",
    "Странная технология приготовления",
    "Странный рис",
    "Только предоплата"
]

problem_embeddings = model.encode(problem_list, convert_to_tensor=True)
problem_embeddings = normalize(problem_embeddings.cpu().numpy(), axis=1)

def match_problems_cosine(review_text):
    review_embedding = model.encode(review_text, convert_to_tensor=True)
    review_embedding = normalize(review_embedding.cpu().numpy().reshape(1, -1))

    similarities = cosine_similarity(review_embedding, problem_embeddings)[0]
    problem_scores = [(problem, similarity) for problem, similarity in zip(problem_list, similarities)]
    sorted_problems = sorted(problem_scores, key=lambda x: x[1], reverse=True)

    threshold = 0.5
    return [problem for problem in sorted_problems if problem[1] > threshold]

# Пример отзыва
review = "доставку я не получила. Два часа ждала"
matched_problems = match_problems_cosine(review)

print("Две наиболее вероятные проблемы:")
for problem, similarity in matched_problems[:2]:
    print(f"- {problem}: {similarity:.2f}")

Две наиболее вероятные проблемы:
- Не предоставили чек: 0.76
- Не доставлен: 0.76


### Выявление причины позитивного отзыва

In [ ]:
from transformers import pipeline

# Загрузка модели для генерации текста на русском языке
generator = pipeline("text2text-generation", model="ai-forever/ruGPT-3.5-13B")

# Функция для объяснения положительных отзывов
def positive_reason(review):
    input_text = f"Почему этот отзыв положительный? Ответь кратко. Вот отзыв: {review}"
    result = generator(input_text, max_length=50, num_return_sequences=1)
    return result[0]['generated_text']

# Пример использования
review = "Было очень вкусно, доставили быстро"
print(positive_reason(review))


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Device set to use cpu
The model 'GPT2LMHeadModel' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'Qwen2AudioForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration']

## Шаг 4. Выделение эмоций

Оптимально использовать 6 основных эмоций: радость, грусть, гнев, страх, удивление, отвращение. Эти эмоции универсальны и покрывают большинство сценариев.

In [78]:
emoberta = pipeline("text-classification", model="tae898/emoberta-base")

def detect_emotion(review):
    emotions = emotion_analyzer(review)
    return max(emotions, key=lambda x: x['score'])['label']  # e.g., 'anger'

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/407 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

## Шаг 5. Генерация рекомендаций

Используем GPT для построения рекомендаций. Рекомендации можно отправлять ресторанам - партнерам для лучшей коммуникации.


In [79]:
def generate_recommendation(problem):
    prompt = f"Provide a recommendation for the problem: {problem}"
    result = generator(prompt, max_length=50)
    return result[0]['generated_text']